# Etudes des valeurs foncières Française
Liens: https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/

In [1]:
# Gather Latitude Longitude from Google Map (time/cost consuming)
add_lat_long = False

In [2]:
import numpy as np
import pandas as pd
import missingno as msno
import glob
from datetime import datetime

pd.options.display.max_columns = None

## Chargement du fichiers des valeurs foncières

In [9]:
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y') # 03/01/2018
def_types = {#'Code postal': 'category',
             #'B/T/Q': 'category',
             #'Nature mutation': 'category',
             #'Type de voie': 'category',
             #'Voie': 'category',
             #'Commune': 'category',
             #'Code departement': 'category',
             #'Type local': 'category',
             #'Nature culture': 'category',
             #'Nature culture speciale': 'category',
             
             'No voie': 'Int64',
             'Surface Carrez du 1er lot':'Float16',
             'Surface Carrez du 2eme lot':'Float16',
             'Surface Carrez du 3eme lot':'Float16',
             'Surface Carrez du 4eme lot':'Float16',
             'Surface Carrez du 5eme lot':'Float16',
             'Nombre de lots':'Int16',
             'Nombre de lots':'Int16',
             'Surface reelle bati': 'Float32',
             'Nombre pieces principales': 'Int8',
             'Surface terrain':'Float32'
            }
col_exclude = ['Code service CH', 'Reference document', '1 Articles CGI',
               '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'Identifiant local',
               'Prefixe de section', 'Section', 'No plan',
               'No Volume','No disposition','Code type local', 'Code voie', 
               '5eme lot', '4eme lot', '3eme lot', '2eme lot', '1er lot']
all_files = glob.glob("data/*.txt")
#df = pd.concat((pd.read_csv(f, sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse, usecols=lambda col: col not in col_exclude) for f in all_files))
df = pd.read_csv('data/valeursfoncieres-2019.txt', sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse,usecols=lambda col: col not in col_exclude)

/Users/odyssee/opt/anaconda3/envs/jupy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (18,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Renomme les colonnes

In [10]:
import unidecode

columns = {}
for index, column_name in enumerate(df.columns):
    columns[column_name] = unidecode.unidecode(column_name.replace(' ','_').replace('\'','_').lower())
df.rename(columns=columns, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017154 entries, 0 to 1017153
Data columns (total 23 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   date_mutation               1017154 non-null  datetime64[ns]
 1   nature_mutation             1017154 non-null  object        
 2   valeur_fonciere             1004369 non-null  float64       
 3   no_voie                     606327 non-null   Int64         
 4   b/t/q                       44359 non-null    object        
 5   type_de_voie                593091 non-null   object        
 6   voie                        1004961 non-null  object        
 7   code_postal                 1004998 non-null  float64       
 8   commune                     1017154 non-null  object        
 9   code_departement            1017154 non-null  object        
 10  code_commune                1017154 non-null  int64         
 11  surface_carrez_du_1er_lo

## Supression des colonnes sans données

On gardes les colonnes sans valeurs foncières car elles peuvent liées à d'autre transaction

In [ ]:
# Suppression des lignes où il n'y a pas de 'valeurs foncières'
#indexNames = df[df['valeur_fonciere'].isnull()].index
#df.drop(indexNames, inplace=True)

## Recherche des doublons

In [ ]:
#df[df.duplicated(keep=False)]

In [15]:
# Drop duplicates
initial_length = len(df)
df.drop_duplicates(inplace=True)
print("Suppression des doublons: {}".format(initial_length-len(df)))

Suppression des doublons: 98514


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918640 entries, 0 to 1017153
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   date_mutation               918640 non-null  datetime64[ns]
 1   nature_mutation             918640 non-null  object        
 2   valeur_fonciere             907760 non-null  float64       
 3   no_voie                     532676 non-null  Int64         
 4   b/t/q                       38718 non-null   object        
 5   type_de_voie                515373 non-null  object        
 6   voie                        912188 non-null  object        
 7   code_postal                 912218 non-null  float64       
 8   commune                     918640 non-null  object        
 9   code_departement            918640 non-null  object        
 10  code_commune                918640 non-null  int64         
 11  surface_carrez_du_1er_lot   89372 non-

#### Transaction en doublons: Même prix, même date

In [17]:
doublons = df.groupby(['date_mutation','valeur_fonciere']).filter(lambda x: len(x) == 1)
#doublons = doublons[doublons.values==1]
doublons

,date_mutation,nature_mutation,valeur_fonciere,no_voie,b/t/q,type_de_voie,voie,code_postal,commune,code_departement,code_commune,surface_carrez_du_1er_lot,surface_carrez_du_2eme_lot,surface_carrez_du_3eme_lot,surface_carrez_du_4eme_lot,surface_carrez_du_5eme_lot,nombre_de_lots,type_local,surface_reelle_bati,nombre_pieces_principales,nature_culture,nature_culture_speciale,surface_terrain
23,2019-01-03,Vente,11592.9,<NA>,NaN,NaN,LES GRANGES,1150.0,SAINT-SORLIN-EN-BUGEY,1,386,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,<NA>,J,NaN,330.0
48,2019-01-09,Vente en l'état futur d'achèvement,716000.0,<NA>,NaN,NaN,BARITET,1280.0,PREVESSIN-MOENS,1,313,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,<NA>,NaN,NaN,NaN
62,2019-01-14,Vente,682000.0,933,NaN,RUE,DE PARIS,1170.0,GEX,1,173,NaN,NaN,NaN,NaN,NaN,0,Maison,142.0,5,S,NaN,2148.0
77,2019-01-07,Vente,779.5,<NA>,NaN,NaN,LE MARTINET,1200.0,BELLEGARDE-SUR-VALSERINE,1,33,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,<NA>,P,NaN,1559.0
79,2019-01-07,Vente,740.0,<NA>,NaN,NaN,LE MARTINET,1200.0,BELLEGARDE-SUR-VALSERINE,1,33,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,<NA>,BT,NaN,1480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017118,2019-06-27,Vente,825600.0,18,NaN,RUE,MALHER,75004.0,PARIS 04,75,104,NaN,62.78125,NaN,NaN,NaN,2,Appartement,62.0,3,NaN,NaN,NaN
1017119,2019-06-27,Vente,324500.0,30,NaN,RUE,DES ROSIERS,75004.0,PARIS 04,75,104,22.078125,NaN,NaN,NaN,NaN,1,Appartement,24.0,2,NaN,NaN,NaN
1017124,2019-06-20,Vente,960000.0,9,NaN,RUE,DE PALESTRO,75002.0,PARIS 02,75,102,82.812500,NaN,NaN,NaN,NaN,3,Appartement,84.0,4,NaN,NaN,NaN
1017130,2019-06-26,Vente,1007640.0,94,NaN,RUE,MONTMARTRE,75002.0,PARIS 02,75,102,76.125000,NaN,NaN,NaN,NaN,2,Appartement,76.0,3,NaN,NaN,NaN


In [18]:
df_maisons = df[(df['type_local']=='Maison')&(df['nature_mutation']=='Vente')]
df_maisons.size

4628612

In [19]:
df_maisons_simple = df_maisons.groupby(['date_mutation','valeur_fonciere']).filter(lambda x: len(x) == 1)
df_maisons_simple.size

1071294

In [20]:
df_maisons_simple

,date_mutation,nature_mutation,valeur_fonciere,no_voie,b/t/q,type_de_voie,voie,code_postal,commune,code_departement,code_commune,surface_carrez_du_1er_lot,surface_carrez_du_2eme_lot,surface_carrez_du_3eme_lot,surface_carrez_du_4eme_lot,surface_carrez_du_5eme_lot,nombre_de_lots,type_local,surface_reelle_bati,nombre_pieces_principales,nature_culture,nature_culture_speciale,surface_terrain
4,2019-04-04,Vente,36000.0,40,NaN,PL,DE LA FONTAINE,1560.0,SAINT-JULIEN-SUR-REYSSOUZE,1,367,NaN,NaN,NaN,NaN,NaN,0,Maison,60.0,3,S,NaN,96.0
15,2019-01-07,Vente,284410.0,2,A,RUE,DU CHAMP DE FOIRE,1200.0,LANCRANS,1,205,NaN,NaN,NaN,NaN,NaN,0,Maison,84.0,4,S,NaN,231.0
24,2019-01-02,Vente,139480.0,5152,NaN,NaN,CHEZ LES LIEVRES,1640.0,ABERGEMENT-DE-VAREY (L ),1,2,NaN,NaN,NaN,NaN,NaN,0,Maison,140.0,5,S,NaN,52.0
33,2019-01-11,Vente,198500.0,34,NaN,CHE,BOLERIN,1150.0,SAINTE-JULIE,1,366,NaN,NaN,NaN,NaN,NaN,0,Maison,100.0,5,S,NaN,658.0
62,2019-01-14,Vente,682000.0,933,NaN,RUE,DE PARIS,1170.0,GEX,1,173,NaN,NaN,NaN,NaN,NaN,0,Maison,142.0,5,S,NaN,2148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014385,2019-04-05,Vente,870000.0,4,NaN,VLA,ALSACE,75019.0,PARIS 19,75,119,NaN,NaN,NaN,NaN,NaN,0,Maison,63.0,4,S,NaN,76.0
1015163,2019-05-24,Vente,975000.0,46,NaN,RUE,DE MOUZAIA,75019.0,PARIS 19,75,119,97.3125,NaN,NaN,NaN,NaN,1,Maison,99.0,4,NaN,NaN,NaN
1015333,2019-05-23,Vente,1098800.0,10,NaN,RUE,IRENEE BLANC,75020.0,PARIS 20,75,120,NaN,NaN,NaN,NaN,NaN,0,Maison,80.0,3,S,NaN,115.0
1015640,2019-06-17,Vente,1030000.0,25,NaN,VLA,DES LILAS,75019.0,PARIS 19,75,119,NaN,NaN,NaN,NaN,NaN,0,Maison,125.0,4,S,NaN,60.0


In [ ]:
df_maisons_simple['surface_carrez'] = df_maisons_simple['sur'].add(df_maisons_simple['Surface Carrez du 2eme lot'], fill_value=0)
df_maisons_simple
# + df_maisons_simple['Surface Carrez du 3eme lot'] + df_maisons_simple['Surface Carrez du 4eme lot'] + df_maisons_simple['Surface Carrez du 5eme lot']





In [ ]:
## from IPython.display import display, HTML

idx = 0
for index, row in doublons.iteritems():#break
    date_mutation = index[0]
    valeur_fonciere = index[1]
    #code_postal = index[2]
    doublon = df[(df['Date mutation']==date_mutation) 
                  & (df['Valeur fonciere']==valeur_fonciere)]
    
#    if doublon[doublon['Type local']=='Maison'].size == doublon.size:
    display(doublon)
    idx +=1
    if idx ==20:break

    
    #if len(trick_df)==0:continue

#### Suppression des données abberrantes

In [ ]:
df.iloc[df.isnull().sum(1).sort_values(ascending=True).index]

#### Analyse

In [ ]:
df.describe()

In [ ]:
df.info(verbose=False, memory_usage="deep")

## Nettoyage des données simple

In [ ]:
def missing_values_assessment(df):
    nan_values = df.isnull().sum().sum()
    print('Nombre d\'observations: {:,}'.format(len(df)))
    print('Nombre de valeurs: {:,}'.format(df.size))
    print('Valeurs manquantes: {:,}'.format(nan_values))
    print('Qualité des données: {}%'.format(100-round((nan_values/df.size)*100,2)))
    print('Type de données:\n {}%'.format(df.dtypes.value_counts()))
    analysis = {'Manquant': df.isnull().sum(),
                'Manquant %':round((df.isnull().sum()/len(df))*100, 2),
                'Type':df.dtypes
               }
    return pd.DataFrame(analysis)

#### Analyse des données manquantes

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

In [ ]:
# Remplissage des mètres Carrez manquant avec zéro
#df = df[df['Nombre de lots']<=5].dropna(how='all', axis=1)
df['surface_carrez_du_5eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_4eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_3eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_2eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_1er_lot'].fillna(0, inplace=True)
df['surface_reelle_bati'].fillna(0, inplace=True)
df['surface_terrain'].fillna(0, inplace=True)
df['nombre_pieces_principales'].fillna(0, inplace=True)

#### Analyse graphique des données manquantes

In [ ]:
msno.bar(df, sort='ascending')

In [ ]:
msno.matrix(df, sort='ascending')

## Nettoyage de données fonctionnelles

In [ ]:
# Colonne 'B/T/Q'
df.loc[df['b/t/q'].isin([',','/','*',"'",'-','.']), ['b/t/q']] = np.NaN

In [ ]:
# Colonne Code Département
df['code_departement'] = df['code_departement'].astype('str')
df.loc[df['code_departement']=='2A', ['code_departement']] = '120'
df.loc[df['code_departement']=='2B', ['code_departement']] = '121'
df['code_departement'] = df['code_departement'].astype(np.int16) # Converti en int16 pour la mémoire

In [ ]:
# Colonne code commune
df['code_commune'] = df['code_commune'].astype(np.int16)

In [ ]:
## Formatage du code postal
df.code_postal=df.code_postal.apply(lambda x: '0{}'.format(x) if len(str(x))==4 else x)

In [ ]:
df.info()

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

In [ ]:
df
df.to_csv(r'data/valeurs_foncieres_excel.csv', index = None, header=True, sep=';')

## Suppression des données  plus utilisés

In [ ]:
# Nous n'avons plus besoin du code postal car nous avons le code commune et le code département
df.drop(['code_postal'],axis=1, inplace=True)

## Sauvegarde des données finales

In [ ]:
df.to_csv(r'data/valeursfoncieres-clean-df.csv', index = None, header=True)